In [3]:
import keras
import pandas as pd

from keras.src.utils import to_categorical
from nltk.stem import SnowballStemmer

import tensorflow as tf
from tensorflow.keras import layers

2024-03-14 11:08:59.643611: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-14 11:08:59.643645: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-14 11:08:59.644429: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-14 11:08:59.649401: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-14 11:09:00.619054: W tensorflow/compiler/tf2

DATA PREPROCESSING

In [4]:
stemmer = SnowballStemmer(language='russian')
data = pd.read_csv("./data/labeled.csv")
data

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0
...,...,...
14407,Вонючий совковый скот прибежал и ноет. А вот и...,1.0
14408,А кого любить? Гоблина тупорылого что-ли? Или ...,1.0
14409,"Посмотрел Утомленных солнцем 2. И оказалось, ч...",0.0
14410,КРЫМОТРЕД НАРУШАЕТ ПРАВИЛА РАЗДЕЛА Т.К В НЕМ Н...,1.0


In [5]:
data = data.where(data.toxic == 1)
data.dropna(inplace=True)
data = data.comment

In [6]:
file = []
with open("./data/dataset.txt", "r") as f:
    file = f.readlines()
import re

toxic_strs = [str for str in file if not re.match("__label__NORMAL", str)]
toxic_strs

['__label__INSULT скотина! что сказать\n',
 '__label__INSULT,__label__THREAT заколоть этого плешивого урода что бы крякнул как селезень гандон штопанный всю вакцину ему в бошку что бы конкретно его завернуло\n',
 '__label__INSULT долбоебы это фэйк\n',
 '__label__INSULT пиздаболы, сделали снимок, придумали историю и подали дурачкам через сми\n',
 '__label__INSULT мляяя..фомин..ты издесь умничаешь,чайка помойная...сука..неужели ты думаешь что уже непопробовали через друзей знвкомых?,\n',
 '__label__INSULT анна склярова (герасименкова) -- защитник я или не не тебе решать, а ты -- точно тупой вонючий тролль и старый пидорас пошёл нах !!!!!!!(tr) \n',
 '__label__INSULT он сразу подписал себе приговор звание - пустослов и балабол, этот чак норис. как он реализует свою обещалку!? да никак. действительно, что вылез всеми позабытый что бы обосраться пустословием.\n',
 '__label__INSULT,__label__OBSCENITY хрен в жопу этому мудаку , а не респект\n',
 '__label__INSULT украина территория населенная 

In [7]:
data = pd.concat([data, pd.Series(toxic_strs)], ignore_index = True)
data.shape

class dataset(keras.utils.Sequence):
    def __init__(self, x, y, batch_size, type):
        self.x, self.y = x, y
        self.batch_size = batch_size
        self.type = type

    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)

    def __getitem__(self, idx):
        l = idx * self.batch_size
        h = min(l + self.batch_size, len(self.x))
        if self.type == "word":
            q = to_categorical(self.y[l:h], num_classes=len(tokens_word.index_word) + 1)
        else:
            q = to_categorical(self.y[l:h], num_classes=len(tokens_suffix.index_word) + 1)
            print(q.shape)
        return np.array(self.x[l:h]), np.array(q)

(49431,)

In [8]:

#CLEANING

pattern_url = r"\bhttps?://\S+\b"
pattern_label = r"__label__[A-Z]*,? ?"
pattern_symbols = r"[^А-Яа-я ]"

for i, string in enumerate(data.values):
    string = re.sub(pattern_url, "", string)
    string = re.sub(pattern_label, "", string) 
    string = re.sub(pattern_symbols, " ", string)
    string = re.sub(r"[ ]+", " ", string)
    data[i] = string

In [9]:
#stemming "data" and save suffixes

data_stemmed = [[] for _ in range(len(data.values))]
suffixes = [[] for _ in range(len(data.values))]

for i, sentence in enumerate(data.values):
    stemmed_words = [stemmer.stem(word) for word in sentence.split()]
    data_stemmed[i] = stemmed_words

    for word in sentence.lower().split():
        s_word = stemmer.stem(word)
        match = re.search(r"" + s_word + r"([а-я]{1,})$", word)
        if match:
            suffixes[i].append(match.group(1))
        else:
            suffixes[i].append("")
    
    

In [10]:
max_words = max([len(arr) for arr in data_stemmed])
max_words

1034

In [11]:
#Tokenize

tokenizer_words = Tokenizer()
tokenizer_suffix = Tokenizer()

tokenizer_words.fit_on_texts(data_stemmed)
tokenizer_suffix.fit_on_texts(suffixes)

tokens_words = tokenizer_words.texts_to_sequences(data_stemmed)
tokens_suffix = tokenizer_suffix.texts_to_sequences(suffixes)


In [12]:
categoricals_word = len(tokenizer_words.index_word)
categoricals_suffix = len(tokenizer_suffix.index_word)

print(categoricals_word), print(categoricals_suffix)

45740
441


(None, None)

V2.0 
words: input shape = (max_words, ) 
suf: words -> last word suffix; input shape (max_words, )

In [11]:
input_seq = []
target_seq = []
target_suf = []
for seq, suf_list in zip(tokens_words, tokens_suffix):
    for i in range(1,len(seq)-1):
       input_seq.append(seq[0:i])
       target_seq.append(seq[i])
       target_suf.append(suf_list[i-1])
       
print(len(target_seq))
print(len(input_seq))
    

608002
608002


In [12]:
input_seq = pad_sequences(input_seq, maxlen=max_words, padding='pre')

In [13]:
BATCH_SIZE = 100
EPOCHS = 20

In [14]:
model_word = keras.Sequential([
    layers.Embedding(categoricals_word, 150, input_shape=(max_words-1,)),
    layers.Bidirectional(layers.LSTM(200)),
    layers.Dense(categoricals_word+1, activation='softmax')
])

model_suf = keras.Sequential([
    layers.Embedding(categoricals_suffix, 150, input_shape=(max_words-1,)),
    layers.Bidirectional(layers.LSTM(200)),
    layers.Dense(categoricals_suffix+1, activation='softmax')
])


2024-03-10 18:05:55.013008: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-10 18:05:55.117656: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-10 18:05:55.117820: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [16]:
from training.data_containers import dataset

word_data = dataset(
    input_seq[0:round(0.9*(len(input_seq)))], 
    target_seq[0:round(0.9*(len(input_seq)))], 
    100, type='word')
test_word = dataset( 
    input_seq[round(0.9*(len(input_seq))):], 
    target_seq[round(0.9*(len(input_seq))):], 
    100, type='word')
suf_data = dataset(
    input_seq[0:round(0.9*(len(input_seq)))], 
    target_suf[0:round(0.9*(len(input_seq)))], 
    100, type='suf')
test_suf = dataset(
    input_seq[round(0.9*(len(input_seq))):],
    target_suf[round(0.9*(len(input_seq))):],
    100, type='suf')
print(input_seq.shape)
print(len(target_suf))

(608002, 1034)
608002


In [17]:
model_word.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [66]:
model_word.fit(
    word_data, 
    validation_data=test_word,
    epochs= EPOCHS,
)

SyntaxError: positional argument follows keyword argument (2470431440.py, line 5)

In [24]:
 categoricals_suffix

441

In [71]:
model_suf = keras.Sequential([
    layers.Embedding(categoricals_suffix+1, 150, input_shape=(max_words-1,)),
    layers.Bidirectional(layers.LSTM(200)),
    layers.Dense(442, activation='softmax')
])

model_suf.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model_suf.fit(
    suf_data,
    validation_data=test_suf,
    epochs= 20,
)

Epoch 1/20
5473/5473 [==============================] - 2525s 461ms/step - loss: 1.8795 - accuracy: 0.5563 - val_loss: 1.6835 - val_accuracy: 0.5821
Epoch 2/20
5473/5473 [==============================] - 2615s 478ms/step - loss: 1.6926 - accuracy: 0.5787 - val_loss: 1.6315 - val_accuracy: 0.5886
Epoch 3/20
5473/5473 [==============================] - 2614s 478ms/step - loss: 1.6523 - accuracy: 0.5848 - val_loss: 1.6082 - val_accuracy: 0.5934
Epoch 4/20
5473/5473 [==============================] - 2594s 474ms/step - loss: 1.6269 - accuracy: 0.5891 - val_loss: 1.5948 - val_accuracy: 0.5966
Epoch 5/20
5473/5473 [==============================] - 2611s 477ms/step - loss: 1.6085 - accuracy: 0.5923 - val_loss: 1.5894 - val_accuracy: 0.5985
Epoch 6/20
5473/5473 [==============================] - 2707s 495ms/step - loss: 1.5927 - accuracy: 0.5952 - val_loss: 1.5827 - val_accuracy: 0.6004
Epoch 7/20
5473/5473 [==============================] - 2580s 471ms/step - loss: 1.5792 - accuracy: 0.5973

In [31]:
model_word.save("toxic_t9_v2_word.h5")
model_suf.save("toxic_t9_v2_suf.h5")

In [27]:
x, y = suf_data[0]
print(x.shape)
print(y.shape)

(100, 1034)
(100, 442)


In [72]:
def transform(input):
    transformed_word = []
    transformed_suf = []
    for prompt in input:
        words = prompt.lower().split()
        transformed_word.append([stemmer.stem(word) for word in words])
        suf = []
        for word in words:
            match = re.search(r"" + stemmer.stem(word) + r"([а-я]{1,})$", word)
            if match:
                suf.append(match.group(1))
            else:
                suf.append("")
        transformed_suf.append(suf)

    transformed_word = tokenizer_words.texts_to_sequences(transformed_word)
    transformed_suf = tokenizer_suffix.texts_to_sequences(transformed_suf)

    transformed_word = pad_sequences(transformed_word, maxlen=max_words-1, padding="pre")
    transformed_suf = pad_sequences(transformed_suf, maxlen=max_words-1, padding="pre")

    return transformed_word, transformed_suf


def predict(transformed_word, transformed_suf, num_words=10):
    predicted_words = []
    predicted_suff = []
    for prompt_word, prompt_suf in zip(transformed_word, transformed_suf):
        prompt_word = np.expand_dims(prompt_word, axis=0)
        prompt_suf = np.expand_dims(prompt_suf, axis=0)

        words = []
        suffs = []
        for _ in range(num_words):
            predicted_word = model_word.predict(prompt_word, verbose=0)
            
            predicted_word = np.argmax(predicted_word)
            words.append(predicted_word)

            predicted_suf = model_suf.predict(prompt_suf, verbose=0)
            
            predicted_suf = np.argmax(predicted_suf)
            suffs.append(predicted_suf)
            
            prompt_word = np.append(prompt_word, predicted_word)
            prompt_word = pad_sequences([prompt_word], maxlen=max_words-1, padding="pre")
            prompt_suf = np.append(prompt_suf, predicted_suf)
            prompt_suf = pad_sequences([prompt_suf], maxlen=max_words-1, padding="pre")

        predicted_words.append(words)
        predicted_suff.append(suffs)

    return predicted_words, predicted_suff


In [73]:
seq, suf = predict(*transform([
    "пидор",
    "привет",
    "еблан",
]), num_words=15)
outputs = ["" for _ in seq]
for j, (sequence, suffixes) in enumerate(zip(seq, suf)):
    
    for i, (word, suf) in enumerate(zip(sequence, suffixes)):
        output_word = ""
        for root, index in tokenizer_words.word_index.items():
            if word == index:
                output_word = root
                break
        for suf_str, index in tokenizer_suffix.word_index.items():
            if suf == index:
                output_word += suf_str
                break
        outputs[j] += output_word + " "
        
for seq in outputs:
    print(seq)

он и в африк не обща не хочет в жизн а те кто их спрашива 
я хоч теб трахнут и тво мыл мыл и на днях скотин вменя отда реб 
ты путинск с сдохнеш преда росс скор так развал стран по всем хват на русск 


In [57]:
tokenizer_suffix.word_index

{'': 1,
 'о': 2,
 'а': 3,
 'и': 4,
 'е': 5,
 'ь': 6,
 'у': 7,
 'ы': 8,
 'ть': 9,
 'я': 10,
 'ой': 11,
 'ов': 12,
 'ить': 13,
 'ая': 14,
 'й': 15,
 'ом': 16,
 'ый': 17,
 'их': 18,
 'ие': 19,
 'ей': 20,
 'ые': 21,
 'л': 22,
 'ого': 23,
 'ю': 24,
 'ит': 25,
 'ое': 26,
 'ли': 27,
 'ия': 28,
 'ил': 29,
 'ий': 30,
 'ет': 31,
 'ии': 32,
 'им': 33,
 'ют': 34,
 'ую': 35,
 'ами': 36,
 'ем': 37,
 'ят': 38,
 'ам': 39,
 'ых': 40,
 'ите': 41,
 'или': 42,
 'ому': 43,
 'ла': 44,
 'ся': 45,
 'ах': 46,
 'ию': 47,
 'ым': 48,
 'его': 49,
 'уй': 50,
 'н': 51,
 'ила': 52,
 'ее': 53,
 'ен': 54,
 'ться': 55,
 'ишь': 56,
 'ется': 57,
 'ешь': 58,
 'но': 59,
 'ится': 60,
 'сь': 61,
 'ись': 62,
 'ев': 63,
 'ему': 64,
 'ью': 65,
 'ими': 66,
 'ям': 67,
 'лся': 68,
 'ный': 69,
 'йте': 70,
 'ья': 71,
 'лись': 72,
 'ло': 73,
 'ются': 74,
 'ыми': 75,
 'ив': 76,
 'иться': 77,
 'ны': 78,
 'ился': 79,
 'на': 80,
 'ные': 81,
 'ье': 82,
 'ете': 83,
 'ует': 84,
 'ная': 85,
 'уют': 86,
 'ься': 87,
 'лась': 88,
 'ием': 89,
 'я

In [67]:
print(target_suf)

[12, 1, 1, 1, 8, 2, 3, 23, 3, 1, 1, 1, 1, 12, 5, 5, 4, 1, 12, 1, 2, 6, 1, 5, 7, 4, 29, 2, 5, 1, 19, 1, 99, 1, 1, 29, 1, 10, 1, 155, 5, 9, 1, 1, 4, 1, 1, 10, 33, 2, 1, 1, 4, 10, 1, 29, 17, 1, 1, 1, 1, 20, 52, 28, 9, 1, 9, 113, 1, 1, 18, 1, 57, 16, 85, 3, 28, 1, 1, 5, 1, 1, 1, 1, 10, 13, 30, 1, 86, 1, 3, 5, 36, 1, 1, 21, 8, 1, 1, 49, 9, 10, 68, 37, 1, 6, 24, 1, 50, 1, 8, 35, 28, 5, 23, 1, 9, 2, 1, 1, 1, 1, 1, 1, 1, 61, 53, 12, 1, 1, 31, 1, 11, 1, 1, 22, 1, 2, 3, 3, 96, 1, 5, 4, 8, 62, 55, 1, 5, 1, 22, 7, 1, 11, 11, 5, 1, 15, 4, 3, 1, 1, 6, 1, 2, 6, 2, 1, 32, 1, 1, 2, 8, 1, 1, 1, 82, 28, 37, 1, 10, 1, 1, 3, 1, 1, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 3, 1, 15, 1, 4, 9, 26, 6, 4, 5, 9, 1, 82, 23, 3, 1, 1, 5, 2, 1, 1, 1, 1, 7, 1, 20, 11, 14, 3, 1, 5, 7, 1, 11, 1, 11, 1, 17, 3, 1, 1, 1, 4, 1, 1, 1, 21, 105, 21, 1, 23, 1, 49, 1, 1, 27, 1, 1, 1, 21, 8, 1, 19, 21, 8, 1, 3, 1, 1, 1, 2, 1, 31, 11, 140, 1, 24, 6, 1, 4, 6, 1, 6, 1, 4, 5, 1, 39, 1, 5, 11, 1, 1, 2, 11, 1, 4, 10, 6, 1, 5, 44, 5, 60, 55,

In [70]:
word = "тюбик"
match = re.search(r"" + stemmer.stem(word) + r"([а-я]{1,})$", word)
match.group(1)

AttributeError: 'NoneType' object has no attribute 'group'

v.2.1 every nn using only max last 10 tokens to generate response

word: input = (10, )
suff: input = (10, )

In [11]:
#data_stemmed
#suffixes

In [13]:
tokens_word = tokenizer_words.texts_to_sequences(data_stemmed)
tokens_suf = tokenizer_suffix.texts_to_sequences(suffixes)

data_tokens_word = []
data_tokens_suf = []
for sequence, suffs in zip(tokens_word, tokens_suf):
    for i in range(2,len(sequence)):
        data_tokens_word.append(sequence[max(0,i-11):i])
        data_tokens_suf.append(suffs[max(0,i-11):i])
        

In [14]:
len(data_tokens_word)

608002

In [15]:
data_tokens_word_x = [s[:-1] for s in data_tokens_word]
data_tokens_word_y = [s[-1:] for s in data_tokens_word]
data_tokens_suf_x = [s[:-1] for s in data_tokens_suf]
data_tokens_suf_y = [s[-1] for s in data_tokens_suf]

In [16]:
import numpy as np
import math

class dataset (tf.keras.utils.Sequence):
    def __init__(self, x, y, batch_size= 200, type = 'w'):
        self.x = x
        self.y = y
        self.batch_size = batch_size
        self.type = type
        
    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)
    
    def __getitem__(self, idx):
        l = idx * self.batch_size
        h = min(l + self.batch_size, len(self.x))
        if self.type == "w":
            q = to_categorical(self.y[l:h], num_classes=categoricals_word+1)
        else:
            q = to_categorical(self.y[l:h], num_classes=categoricals_suffix+1)
        return np.array(pad_sequences(self.x[l:h], maxlen=10, padding='pre')), np.array(q)
        

In [17]:
d_word = dataset(data_tokens_word_x, data_tokens_word_y, batch_size= 150)

In [18]:
model_word_10inputs = keras.models.load_model('toxic_t9_word_v2_1.keras')

2024-03-14 11:10:33.814073: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-14 11:10:33.839839: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-14 11:10:33.840019: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [19]:
#model_word_10inputs = keras.Sequential([
#    layers.Embedding(categoricals_word, 150, input_shape=(10,)),
#   layers.Bidirectional(layers.LSTM(200)),
#    layers.Dense(categoricals_word+1, activation='softmax')
#])

#model_word_10inputs.compile(
#    optimizer='Adam',
#    loss='categorical_crossentropy',
#    metrics=['accuracy']
#)

model_word_10inputs.fit(
    d_word,
    epochs=15
)

Epoch 1/15


2024-03-14 11:10:40.026773: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-03-14 11:10:40.095277: I external/local_xla/xla/service/service.cc:168] XLA service 0x73f749072040 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-14 11:10:40.095305: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2024-03-14 11:10:40.100233: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1710403840.181924   11114 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


4054/4054 [==============================] - 278s 68ms/step - loss: 1.1475 - accuracy: 0.7427
Epoch 2/15
4054/4054 [==============================] - 308s 76ms/step - loss: 1.1430 - accuracy: 0.7430
Epoch 3/15
4054/4054 [==============================] - 350s 86ms/step - loss: 1.1352 - accuracy: 0.7447
Epoch 4/15
4054/4054 [==============================] - 391s 96ms/step - loss: 1.1327 - accuracy: 0.7445
Epoch 5/15
4054/4054 [==============================] - 447s 110ms/step - loss: 1.1265 - accuracy: 0.7459
Epoch 6/15
 100/4054 [..............................] - ETA: 7:25 - loss: 0.9709 - accuracy: 0.7833

KeyboardInterrupt: 

In [32]:
model_word_10inputs.save('toxic_t9_word_v2_1.keras')

In [ ]:
model